<a href="https://colab.research.google.com/github/wzz010624/Scigraphexp/blob/main/full_graph_building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pickle
#coauthor_dict[author_id][coauthor_id][paper_year] = [paper_id]
# 保存为二进制文件
with open('/content/drive/MyDrive/GNNdataset/merged_network_with_feature', 'rb') as f:
    merged_network = pickle.load(f)

In [ ]:
import pickle
import numpy as np
with open('/content/drive/MyDrive/GNNdataset/author_h_index_sequence.pickle', 'rb') as f:
    author_h_index_sequence = pickle.load(f)
# 保存为二进制文件
with open('/content/drive/MyDrive/GNNdataset/authors_high_impact_sequence.pickle', 'rb') as f:
    authors_high_impact_sequence = pickle.load(f)

# 保存为二进制文件
with open('/content/drive/MyDrive/GNNdataset/author_publication_sequence.pickle', 'rb') as f:
    author_publication_sequence = pickle.load(f)

# 保存为二进制文件
with open('/content/drive/MyDrive/GNNdataset/author_citation_sequence.pickle', 'rb') as f:
    author_citation_sequence = pickle.load(f)

In [ ]:
import pickle
#coauthor_dict[author_id][coauthor_id][paper_year] = [paper_id]
# 保存为二进制文件
with open('/content/drive/MyDrive/GNNdataset/target_authors_by_year', 'rb') as f:
    target_authors_by_year = pickle.load(f)

In [ ]:
# 计算所有目标节点和目标引文
all_target_nodes = []
all_target_citations = []
for year, target_authors in target_authors_by_year.items():
    for author_id in target_authors:
        publication_years = sorted(author_publication_sequence[author_id].keys())
        career_start_year = publication_years[0]
        career_end_year = publication_years[-1]
        target_year = career_start_year + 14
        accumulated_citations = sum([author_citation_sequence[author_id].get(year, 0) for year in range(career_start_year, target_year+1)])
        all_target_nodes.append(f"{author_id}_{year}")
        all_target_citations.append(accumulated_citations)

In [ ]:
import torch
def edge_list_to_index_tensor(edge_list, node_id_to_int):
    index_list = [(node_id_to_int[src], node_id_to_int[dst]) for src, dst in edge_list]
    index_tensor = torch.tensor(index_list, dtype=torch.long).t().contiguous()
    return index_tensor
def extract_edge_features(edge_list, network):
    edge_features = []
    for src, dst in edge_list:
        edge_data = network.edges[src, dst]
        edge_features.append([edge_data['total_collaboration_count'], edge_data['collaboration_stability']])
    return edge_features

from torch_geometric.data import Data

def network_to_pyg_data(network, target_nodes, target_citations):
    node_id_to_int = {node_id: i for i, node_id in enumerate(network.nodes)}

    node_features = []
    for node_id in network.nodes:
        node_features.append(list(network.nodes[node_id].values()))
    node_features = torch.tensor(node_features, dtype=torch.float)

    edge_index = edge_list_to_index_tensor(list(network.edges), node_id_to_int)
    edge_features = extract_edge_features(list(network.edges), network)
    edge_attr = torch.tensor(edge_features, dtype=torch.float)

    y = torch.tensor(target_citations, dtype=torch.float)
    mask = torch.tensor([node_id in target_nodes for node_id in network.nodes], dtype=torch.bool)
    return Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, y=y, mask=mask)

def create_train_test_masks(target_nodes, target_citations, train_ratio=0.8):
    num_nodes = len(target_nodes)
    num_train = int(num_nodes * train_ratio)
    
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)
    
    indices = torch.randperm(num_nodes)
    train_indices = indices[:num_train]
    test_indices = indices[num_train:]
    
    train_mask[train_indices] = True
    test_mask[test_indices] = True
    
    return train_mask, test_mask

In [ ]:
# 使用合并后的网络和所有目标节点以及目标引文创建 PyTorch Geometric 数据对象
ppyg_data = network_to_pyg_data(merged_network, all_target_nodes, all_target_citations)

In [ ]:
ppyg_data

Data(x=[726152, 25], edge_index=[2, 2494234], edge_attr=[2494234, 2], y=[92623], mask=[726152])

In [ ]:
ppyg_data.x[ppyg_data.mask][0]

tensor([1.0000, 0.2000, 4.0000, 0.8000, 1.0000, 0.0000, 0.0000, 1.0000, 0.2000,
        1.0000, 0.2000, 4.0000, 0.8000, 1.0000, 0.0000, 0.0000, 1.0000, 0.2000,
        5.0000, 0.0000, 0.3000, 0.4111, 0.3000, 0.0000, 0.0000])

In [ ]:
ppyg_data.y

tensor([ 62., 231.,  21.,  ...,  37.,   5.,   1.])

In [ ]:
# 计算所有目标节点和目标引文
all_target_nodes = []
all_target_publication = []
all_target_citations = []
all_target_high_impact = []
all_target_h_index = []
all_target_variables = []
for year, target_authors in target_authors_by_year.items():
    for author_id in target_authors:
        publication_years = sorted(author_publication_sequence[author_id].keys())
        career_start_year = publication_years[0]
        career_end_year = publication_years[-1]
        target_year = career_start_year + 14
        accumulated_publication = sum([author_publication_sequence[author_id].get(year, 0) for year in range(career_start_year, target_year+1)])
        accumulated_citations = sum([author_citation_sequence[author_id].get(year, 0) for year in range(career_start_year, target_year+1)])
        accumulated_high_impact = sum([authors_high_impact_sequence[author_id].get(year, 0) for year in range(career_start_year, target_year+1)])
        all_target_nodes.append(f"{author_id}_{year}")
        all_target_citations.append(accumulated_citations)
        all_target_publication.append(accumulated_publication)
        all_target_h_index.append(author_h_index_sequence[author_id].get(target_year, 0))
        all_target_variables.append([accumulated_publication, accumulated_citations, accumulated_high_impact, author_h_index_sequence[author_id].get(target_year, 0)])

In [ ]:
import torch
def edge_list_to_index_tensor(edge_list, node_id_to_int):
    index_list = [(node_id_to_int[src], node_id_to_int[dst]) for src, dst in edge_list]
    index_tensor = torch.tensor(index_list, dtype=torch.long).t().contiguous()
    return index_tensor
def extract_edge_features(edge_list, network):
    edge_features = []
    for src, dst in edge_list:
        edge_data = network.edges[src, dst]
        edge_features.append([edge_data['total_collaboration_count'], edge_data['collaboration_stability']])
    return edge_features

In [ ]:
from torch_geometric.data import Data

def create_train_test_masks(target_nodes, target_citations, train_ratio=0.8):
    num_nodes = len(target_nodes)
    num_train = int(num_nodes * train_ratio)
    
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)
    
    indices = torch.randperm(num_nodes)
    train_indices = indices[:num_train]
    test_indices = indices[num_train:]
    
    train_mask[train_indices] = True
    test_mask[test_indices] = True
    
    return train_mask, test_mask

In [ ]:
import torch
import numpy as np

def get_neighbors(merged_network, node_id):
    neighbors = list(merged_network.neighbors(node_id))
    return neighbors

def compute_features_average(merged_network, neighbors):
    features = [np.array(list(merged_network.nodes[neighbor].values())) for neighbor in neighbors]
    return np.mean(features, axis=0)

def compute_edge_features_average(merged_network, node_id, neighbors):
    edge_features = [np.array(list(merged_network.edges[node_id, neighbor].values())) for neighbor in neighbors]
    return np.mean(edge_features, axis=0)

In [ ]:
from torch_geometric.data import Data
from typing import List

def extract_subgraph(merged_network, target_node: str):
    # Get the neighbors of the target node
    neighbors = get_neighbors(merged_network, target_node)
    
    # Include the target node in the list
    nodes = [target_node] + neighbors
    
    # Create subgraph
    subgraph = merged_network.subgraph(nodes)
    
    return subgraph
    
def subgraph_to_pyg_data(subgraph, target_node, target_variables):
    # The same code as before, until the definition of y.
    node_features = []
    subgraph_node_id_to_int = {}
    
    # Ensure that the target_node is always the first node in the subgraph.nodes list
    nodes = [target_node] + [node for node in subgraph.nodes if node != target_node]
    
    for i, merged_node_id in enumerate(nodes):
        node_features.append(list(subgraph.nodes[merged_node_id].values()))
        subgraph_node_id_to_int[merged_node_id] = i

    node_features = torch.tensor(node_features, dtype=torch.float)
    edge_index = edge_list_to_index_tensor(list(subgraph.edges), subgraph_node_id_to_int)
    edge_features = extract_edge_features(list(subgraph.edges), subgraph)
    edge_attr = torch.tensor(edge_features, dtype=torch.float)
    y = torch.tensor(target_variables, dtype=torch.float)

    data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, y=y)
    return data

In [ ]:
from torch_geometric.data import Data
from typing import List
import pandas as pd

def create_subgraph_pyg_data_list_and_df(merged_network, target_nodes: List[str], target_variables: List[List[float]]):
    subgraph_pyg_data_list = []
    machine_learning_data = []

    merged_node_id_to_int = {merged_node_id: i for i, merged_node_id in enumerate(merged_network.nodes)}

    # The same code as before, until the for loop.
    for i, target_node in enumerate(target_nodes):
        subgraph = extract_subgraph(merged_network, target_node)
        # Pass all target variables to the subgraph_to_pyg_data function.
        subgraph_pyg_data = subgraph_to_pyg_data(subgraph, target_node, target_variables[i])
        subgraph_pyg_data_list.append(subgraph_pyg_data)

        # Compute average features for neighbors
        neighbors = get_neighbors(merged_network, target_node)
        neighbor_features_avg = compute_features_average(merged_network, neighbors)
        edge_features_avg = compute_edge_features_average(merged_network, target_node, neighbors)

        # Get target node features
        target_node_features = subgraph_pyg_data.x[0].numpy()

        # Add all target variables to the row.
        row = [target_node, *target_node_features, *neighbor_features_avg, *edge_features_avg, target_variables[i]]
        machine_learning_data.append(row)
    # The rest of the function stays the same.
    # Create DataFrame
    columns = ["merged_node_id"] + [f"node_feature_{i}" for i in range(target_node_features.shape[0])] + \
              [f"neighbor_feature_{i}" for i in range(target_node_features.shape[0])] + \
              [f"edge_feature_{i}" for i in range(subgraph_pyg_data.edge_attr.shape[1])] + ["y"]
    machine_learning_df = pd.DataFrame(machine_learning_data, columns=columns)

    return subgraph_pyg_data_list, machine_learning_df

  
subgraph_pyg_data_list, machine_learning_df = create_subgraph_pyg_data_list_and_df(merged_network, all_target_nodes, all_target_variables)

In [ ]:
ppyg_data

Data(x=[726152, 25], edge_index=[2, 2494234], edge_attr=[2494234, 2], y=[92623], mask=[726152])

In [ ]:
subgraph_pyg_data_list

[Data(x=[10, 25], edge_index=[2, 13], edge_attr=[13, 2], y=[4]),
 Data(x=[2, 25], edge_index=[2, 1], edge_attr=[1, 2], y=[4]),
 Data(x=[7, 25], edge_index=[2, 21], edge_attr=[21, 2], y=[4]),
 Data(x=[2, 25], edge_index=[2, 1], edge_attr=[1, 2], y=[4]),
 Data(x=[4, 25], edge_index=[2, 4], edge_attr=[4, 2], y=[4]),
 Data(x=[5, 25], edge_index=[2, 7], edge_attr=[7, 2], y=[4]),
 Data(x=[2, 25], edge_index=[2, 1], edge_attr=[1, 2], y=[4]),
 Data(x=[6, 25], edge_index=[2, 6], edge_attr=[6, 2], y=[4]),
 Data(x=[5, 25], edge_index=[2, 5], edge_attr=[5, 2], y=[4]),
 Data(x=[2, 25], edge_index=[2, 1], edge_attr=[1, 2], y=[4]),
 Data(x=[5, 25], edge_index=[2, 5], edge_attr=[5, 2], y=[4]),
 Data(x=[5, 25], edge_index=[2, 4], edge_attr=[4, 2], y=[4]),
 Data(x=[2, 25], edge_index=[2, 1], edge_attr=[1, 2], y=[4]),
 Data(x=[3, 25], edge_index=[2, 2], edge_attr=[2, 2], y=[4]),
 Data(x=[2, 25], edge_index=[2, 1], edge_attr=[1, 2], y=[4]),
 Data(x=[6, 25], edge_index=[2, 10], edge_attr=[10, 2], y=[4]),
 

In [ ]:
machine_learning_df

,merged_node_id,node_feature_0,node_feature_1,node_feature_2,node_feature_3,node_feature_4,node_feature_5,node_feature_6,node_feature_7,node_feature_8,...,neighbor_feature_18,neighbor_feature_19,neighbor_feature_20,neighbor_feature_21,neighbor_feature_22,neighbor_feature_23,neighbor_feature_24,edge_feature_0,edge_feature_1,y
0,53f42c89dabfaee02ac546a6_1971,7.0,1.4,26.0,5.2,3.0,0.0,0.0,9.0,1.8,...,3.888889,0.103783,0.374895,0.368946,0.211905,0.000000,0.351167,1.222222,0.074074,"[10, 62, 0, 5]"
1,53f4b4d4dabfaedd74eba537_1971,5.0,1.0,15.0,3.0,3.0,1.0,0.2,1.0,0.2,...,13.000000,0.164634,0.325000,0.414021,0.204545,0.166667,0.400000,1.000000,0.000000,"[20, 231, 1, 9]"
2,53f49871dabfaeb6ca77baf9_1971,3.0,0.6,1.0,0.2,1.0,0.0,0.0,6.0,1.2,...,5.000000,0.000000,0.190476,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,"[15, 21, 0, 4]"
3,5405d03adabfae8faa60391f_1971,4.0,0.8,41.0,8.2,4.0,1.0,0.2,1.0,0.2,...,5.000000,0.000000,0.300000,0.411111,0.300000,0.000000,0.000000,1.000000,0.000000,"[22, 445, 5, 11]"
4,54484651dabfae87b7dfc041_1971,5.0,1.0,5.0,1.0,1.0,0.0,0.0,3.0,0.6,...,6.666667,0.140760,0.392451,0.439509,0.251984,0.083333,0.300095,1.000000,0.000000,"[34, 267, 0, 10]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92618,53f436e6dabfaec22ba859b3_2011,1.0,0.2,2.0,0.4,1.0,0.0,0.0,2.0,0.4,...,8.000000,0.244180,0.636396,0.745867,0.459387,0.244444,0.564177,1.000000,0.000000,"[28, 71, 1, 5]"
92619,560a6b8a45cedb33970da289_2011,1.0,0.2,0.0,0.0,0.0,0.0,0.0,1.0,0.2,...,8.000000,0.422062,0.656387,0.537739,0.451186,0.261719,0.571485,1.000000,0.000000,"[11, 12, 0, 2]"
92620,53f461e9dabfaee2a1d9d001_2011,2.0,0.4,11.0,2.2,2.0,0.0,0.0,4.0,0.8,...,13.750000,0.334651,0.585457,0.694957,0.431717,0.000000,0.525198,1.250000,0.250000,"[10, 37, 0, 4]"
92621,562528d845cedb339858ac40_2011,1.0,0.2,0.0,0.0,0.0,0.0,0.0,1.0,0.2,...,16.000000,0.365135,0.729194,0.770638,0.507078,0.341270,0.662288,1.000000,0.000000,"[19, 5, 0, 2]"


In [ ]:
import pickle

# 创建一个字典，将对象与对应的键关联起来
data = {
    'ppyg_data': ppyg_data,
    'subgraph_pyg_data_list': subgraph_pyg_data_list,
    'machine_learning_df': machine_learning_df
}

# 将数据保存到 "citations.pickle" 文件中
with open('/content/drive/MyDrive/Experiment_graph/citations.pickle', 'wb') as file:
    pickle.dump(data, file)

In [ ]:
import torch
import pickle
from torch.utils.data import Dataset
with open('/content/drive/MyDrive/raw_train_test_split_data_full4dimension.pkl', 'rb') as f:
  data = pickle.load(f)

In [ ]:
train_machine_learning_df = data['train_machine_learning_df']
test_machine_learning_df = data['test_machine_learning_df']
train_subgraph_pyg_data_list = data['train_subgraph_pyg_data_list']
test_subgraph_pyg_data_list = data['test_subgraph_pyg_data_list']